In [ ]:
!pip install nsepython

In [ ]:
from nsepython import *
from datetime import date

In [ ]:
import pandas as pd
from os import remove
import requests
from urllib.parse import quote, urlencode, urlparse
import datetime, calendar
import json
from requests.exceptions import ReadTimeout
import pickle
from zipfile import BadZipFile, ZipFile
from pathlib import Path
from requests.exceptions import ReadTimeout
import os
import itertools
import pandas as pd
from math import sqrt, log, exp
from scipy.stats import norm

In [ ]:
date = date.today()
date.strftime("%b").upper()

'MAR'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
curdir= Path('/content/drive/MyDrive')

Mounted at /content/drive


**Getting the derivatives data for CTIX relevant order volumes, prices**

In [ ]:
headers = {
    "accept": "*/*",
    "referer": "https://www.nseindia.com/all-reports-derivatives",
    "sec-ch-ua": "\"Not:A-Brand\";v=\"24\", \"Chromium\";v=\"134\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Linux\"",
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "x-requested-with": "XMLHttpRequest"
}

In [ ]:
def parse_cookie_string(cookie_str: str) -> dict:
    cookies = {}
    for cookie in cookie_str.split(";"):
        cookie = cookie.strip()
        if not cookie:
            continue
        try:
            key, value = cookie.split("=", 1)
            cookies[key] = value
        except ValueError:
            # If there is no "=" in the cookie string, skip it or handle accordingly.
            continue
    return cookies
cookiestr= '''
nseQuoteSymbols=[{"symbol":"NIFTYIT","identifier":"","type":"equity"}]
'''
cookies= parse_cookie_string(cookiestr)
cookies

{'nseQuoteSymbols': '[{"symbol":"NIFTYIT","identifier":"","type":"equity"}]'}

In [ ]:
response= requests.get(url= "https://www.nseindia.com/api/reports", stream= True, timeout= 5, headers= headers, cookies= cookies,
                       params= {
                           'archives': '[{"name":"F&O - UDiFF Common Bhavcopy Final (zip)","type":"archives","category":"derivatives","section":"equity"}]',
                           'date': '12-Apr-2022',
                           'type': 'equity',
                           'mode': 'single'
                        })
response

<Response [200]>

In [ ]:
url= "https://www.nseindia.com/api/reports"
#query= "?archives=%5B%7B%22name%22%3A%22F%26O%20-%20UDiFF%20Common%20Bhavcopy%20Final%20(zip)%22%2C%22type%22%3A%22archives%22%2C%22category%22%3A%22derivatives%22%2C%22section%22%3A%22equity%22%7D%5D&date=09-Jan-2025&type=equity&mode=single"


#archives= '[{"name":"F&O - UDiFF Common Bhavcopy Final (zip)","type":"archives","category":"derivatives","section":"equity"}]'
#typeeee= "equity"
#mode= "single"
#date= "09-Jan-2025"

df_list_bhavcopy = []
df_list_udiff = []

def get_data():

    # makes csv files in ./derivaties/__year_name__/
    """
    change year and month to desired
    and change cookie to a fresh cookie by sniffing the request header in chrome network panel
    """
    # change the below lines to get different years' diff months data like this -
    # for years 2021 and 2022's data, this can be done ->  yr = ['2021', '2022']
    # for selecting only jan and feb, this can be done ->  mnth = ['JAN','FEB']
    #yr = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
    #mnth = [4, 5, 6, 7, 8, 9, 10, 11, 12] # some

    #yr = list(range(2005, 2024))
    yr= [2025]
    #mnth = [0] # all
    #mnth = list(range(3, 13)) # from march
    mnth= [1, 2, 3]


    BASE_DIR = curdir
    derivatives = BASE_DIR / 'derivatives'

    alldates = []
    for each_year in yr:
        yr_path = derivatives / str(each_year)
        yr_path.mkdir(parents=True, exist_ok=True)
        if mnth == [0]:
            for each_mnth in range(1, 13):
                num_days = calendar.monthrange(each_year, each_mnth)[1]
                alldates += [datetime.date(each_year, each_mnth, day) for day in range(1, num_days+1)]
        else:
            for each_mnth in mnth:
                num_days = calendar.monthrange(each_year, each_mnth)[1]
                alldates += [datetime.date(each_year, each_mnth, day) for day in range(1, num_days+1)]


    derivatives.mkdir(parents=True, exist_ok=True)

    base_url = url

    for day in alldates:

        cur_year = str(day.year)
        cur_date = ""
        if day.day < 10:
            cur_date += "0"
        cur_date += str(day.day)
        cur_month = day.strftime("%c").split()[1]

        params = {
                'archives': '',
                'date': '{}-{}-{}'.format(cur_date, cur_month, cur_year),
                'type': 'equity',
                'mode': 'single'
                }

        if day.year < 2024:
          params['archives']= '[{"name":"F&O - Bhavcopy(csv)","type":"archives","category":"derivatives","section":"equity"}]'
          csv_name = "fo{}bhav.csv".format(day.strftime("%d%b%Y").upper())
          #cookies= ''
        else:
          params['archives']= '[{"name":"F&O - UDiFF Common Bhavcopy Final (zip)","type":"archives","category":"derivatives","section":"equity"}]'
          csv_name = "BhavCopy_NSE_FO_0_0_0_{}{}{}_F_0000.csv".format(*((day.isoformat()).split('-')))
          print('csv name is: ', csv_name)

        print("downloading day {} rn".format(day.isoformat()))
        cur_year_path = derivatives / cur_year

        day_zipfile_path = cur_year_path / 'day{}.zip'.format(day.isoformat())
        try:
            print(base_url, params)
            response = requests.get(base_url, timeout=5, stream = True, headers= headers, cookies= cookies, params= params)
            print("final request was - ", response.url)
            #response = requests.get(st1, allow_redirects=True, timeout=5)
            with open(day_zipfile_path, 'wb') as ziphandle:
                ziphandle.write(response.content)
        except ReadTimeout:
            print("\ttimed out\n")
            continue

        with open(day_zipfile_path, 'rb') as f:
            try:
                z = ZipFile(f)
                z.extractall(path = cur_year_path)
                print("day {} downloaded".format(day.isoformat()))
                # extract info out of all the csv files extracted in the folder

                csv_err= False
                if day.year < 2024:
                  try:
                    bhavdf= pd.read_csv(cur_year_path / csv_name)
                    bhavdf= bhavdf[(bhavdf['SYMBOL'] == 'INFY') | (bhavdf['SYMBOL'] == 'BEL') | (bhavdf['SYMBOL'] == 'LT') | (bhavdf['SYMBOL'] == 'TCS') | (bhavdf['SYMBOL'] == 'BHARTIARTL') | (bhavdf['SYMBOL'] == 'NTPC') | (bhavdf['SYMBOL'] == 'HCLTECH')]
                    df_list_bhavcopy.append(bhavdf)
                  except FileNotFoundError:
                    csv_err= True
                    print("\tno bhavcopy found\n")
                else:
                  udiff_df= pd.read_csv(cur_year_path / csv_name)
                  udiff_df= udiff_df[(udiff_df['TckrSymb'] == 'INFY') | (udiff_df['TckrSymb'] == 'BEL') | (udiff_df['TckrSymb'] == 'LT') | (udiff_df['TckrSymb'] == 'TCS') | (udiff_df['TckrSymb'] == 'BHARTIARTL') | (udiff_df['TckrSymb'] == 'NTPC') | (udiff_df['TckrSymb'] == 'HCLTECH')]
                  df_list_udiff.append(udiff_df)
                print("day {} extracted".format(day.isoformat()))
                if not csv_err:
                  remove(cur_year_path / csv_name)
            except BadZipFile:
                print("\tbad zip file\n")

        remove(day_zipfile_path)

    #df_bhavcopy= pd.concat(df_list_bhavcopy)
    #jdf_bhavcopy.to_csv(derivatives/ "bhavcopy.csv", index= False)
    df_udiff= pd.concat(df_list_udiff)
    df_udiff.to_csv(derivatives / "udiff.csv", index= False)



In [ ]:
get_data()

csv name is:  BhavCopy_NSE_FO_0_0_0_20250101_F_0000.csv
downloading day 2025-01-01 rn
https://www.nseindia.com/api/reports {'archives': '[{"name":"F&O - UDiFF Common Bhavcopy Final (zip)","type":"archives","category":"derivatives","section":"equity"}]', 'date': '01-Jan-2025', 'type': 'equity', 'mode': 'single'}
final request was -  https://www.nseindia.com/api/reports?archives=%5B%7B%22name%22%3A%22F%26O+-+UDiFF+Common+Bhavcopy+Final+%28zip%29%22%2C%22type%22%3A%22archives%22%2C%22category%22%3A%22derivatives%22%2C%22section%22%3A%22equity%22%7D%5D&date=01-Jan-2025&type=equity&mode=single
day 2025-01-01 downloaded
day 2025-01-01 extracted
csv name is:  BhavCopy_NSE_FO_0_0_0_20250102_F_0000.csv
downloading day 2025-01-02 rn
https://www.nseindia.com/api/reports {'archives': '[{"name":"F&O - UDiFF Common Bhavcopy Final (zip)","type":"archives","category":"derivatives","section":"equity"}]', 'date': '02-Jan-2025', 'type': 'equity', 'mode': 'single'}
final request was -  https://www.nseindi

In [ ]:
unknown_bhavcopy= pd.read_csv(curdir/ "derivatives" / "bhavcopy.csv")
unknown_bhavcopy

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP,Unnamed: 15
0,FUTSTK,BEL,29-Mar-2023,0.0,XX,95.25,97.55,95.25,97.40,97.40,2666,14711.47,98051400,1077300,01-MAR-2023,NaN
1,FUTSTK,BEL,27-Apr-2023,0.0,XX,96.45,98.15,96.25,98.10,98.10,113,627.76,1926600,85500,01-MAR-2023,NaN
2,FUTSTK,BEL,25-May-2023,0.0,XX,96.75,98.45,96.75,98.45,98.45,25,139.80,148200,74100,01-MAR-2023,NaN
3,FUTSTK,BHARTIARTL,29-Mar-2023,0.0,XX,748.60,758.60,745.80,755.55,755.55,5822,41647.34,43135700,-550050,01-MAR-2023,NaN
4,FUTSTK,BHARTIARTL,27-Apr-2023,0.0,XX,752.55,763.30,751.50,760.30,760.30,443,3188.63,456950,112100,01-MAR-2023,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419320,OPTSTK,TCS,28-Mar-2024,4120.0,PE,0.00,0.00,0.00,333.90,316.75,0,0.00,0,0,29-DEC-2023,NaN
419321,OPTSTK,TCS,28-Mar-2024,4140.0,PE,0.00,0.00,0.00,348.20,331.90,0,0.00,0,0,29-DEC-2023,NaN
419322,OPTSTK,TCS,28-Mar-2024,4160.0,PE,0.00,0.00,0.00,362.95,347.35,0,0.00,0,0,29-DEC-2023,NaN
419323,OPTSTK,TCS,28-Mar-2024,4180.0,PE,0.00,0.00,0.00,377.80,363.10,0,0.00,0,0,29-DEC-2023,NaN


**Preprocessing the options data and feature engg**

**Fetching the Sentiments of the week for the entire period**

In [ ]:
!pip install tweepy textblob --quiet

In [ ]:
bearer_token= ""
import tweepy
client= tweepy.Client(bearer_token)
client

In [ ]:
from datetime import datetime, timezone, timedelta

end_time= datetime(2025, 3, 16, tzinfo=timezone.utc)
end_time

In [ ]:
start_time = end_time - timedelta(days=6)
start_time

datetime.datetime(2025, 1, 7, 0, 0, tzinfo=datetime.timezone.utc)

In [ ]:
end_time= end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
start_time= start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
print(end_time, start_time)

2025-03-08T00:00:00Z 2025-01-07T00:00:00Z


In [ ]:
query = "nuclear funding startup funding and breakthroughs"

response = client.search_recent_tweets(
        query=query,
        start_time=start_time,
        end_time=end_time,
        tweet_fields=["created_at", "text", "author_id"], #Optional, choose which fields you want.
        max_results=100 #max is 100 for recent search.
    )
response

BadRequest: 400 Bad Request
Invalid 'start_time':'2025-01-07T00:00Z'. 'start_time' must be on or after 2025-03-09T18:21Z
Invalid 'end_time':'2025-03-08T00:00Z'. 'end_time' must be on or after 2025-03-09T18:21Z

In [ ]:
end_date = date.today().strftime("%d-%m-%Y")
end_date = str(end_date)

#start_date = (datetime.datetime.now()- datetime.timedelta(days=65)).strftime("%d-%m-%Y")
start_date= date(2025, 3, 1).strftime("%d-%m-%Y")
start_date = str(start_date)


In [ ]:
nse_quote_ltp("NIFTY IT")

KeyError: None

In [ ]:
symbol = "CTIX"
start_date = "15-05-2021"
end_date ="15-06-2021"
instrumentType = "options"
expiry_date ="24-Jun-2021"
strikePrice = ""
optionType=""
print(derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,optionType))

Empty DataFrame
Columns: []
Index: []


In [ ]:
symbol = "NIFTY IT"
df = index_pe_pb_div(symbol,start_date,end_date)
df

,RequestNumber,Index Name,pe,pb,divYield,DATE
0,pepb6387735920599397290,Nifty IT,28.35,7.73,2.74,11 Mar 2025
1,pepb6387735920599397290,Nifty IT,28.53,7.78,2.72,10 Mar 2025
2,pepb6387735920599397290,Nifty IT,28.66,7.82,2.71,07 Mar 2025
3,pepb6387735920599397290,Nifty IT,28.91,7.89,2.69,06 Mar 2025
4,pepb6387735920599397290,Nifty IT,28.86,7.87,2.69,05 Mar 2025
5,pepb6387735920599397290,Nifty IT,28.25,7.71,2.75,04 Mar 2025
6,pepb6387735920599397290,Nifty IT,28.51,7.78,2.72,03 Mar 2025
7,pepb6387735920599397290,Nifty IT,28.28,7.72,2.75,28 Feb 2025
8,pepb6387735920599397290,Nifty IT,29.52,8.05,2.63,27 Feb 2025
9,pepb6387735920599397290,Nifty IT,29.66,8.09,2.62,25 Feb 2025


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [ ]:
# Define the symbol and other parameters
symbol = "NIFTY"  # Replace with your desired symbol
option_type = "CE"  # Replace with "PE" for put options
strike_price = 10000  # Replace with your desired strike price
expiry_date = date(2023, 12, 28)  # Replace with your desired expiry date


In [ ]:

# Fetch the data
data = get_history(
    symbol=symbol,
    start=start_date,
    end=end_date,
    option_type=option_type,
    strike_price=strike_price,
    expiry_date=expiry_date
)
data